In [1]:
import time
import requests
from bs4 import BeautifulSoup
import re
import sklearn
import time
import numpy as np
import json
import pandas as pd
pd.set_option("max_colwidth", 200)

from pandarallel import pandarallel
pandarallel.initialize()
# import os

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
base_url =  'https://www.rescue.org/press-release/list'
df = pd.DataFrame([base_url + '?page=%s'%i for i in range(0,71)],  columns =['pages'] )
df

,pages
0,https://www.rescue.org/press-release/list?page=0
1,https://www.rescue.org/press-release/list?page=1
2,https://www.rescue.org/press-release/list?page=2
3,https://www.rescue.org/press-release/list?page=3
4,https://www.rescue.org/press-release/list?page=4
...,...
66,https://www.rescue.org/press-release/list?page=66
67,https://www.rescue.org/press-release/list?page=67
68,https://www.rescue.org/press-release/list?page=68
69,https://www.rescue.org/press-release/list?page=69


In [3]:
def extract_urls(url):
    coverpage = requests.get(url).content
    soup = BeautifulSoup(coverpage, 'html.parser')
    rows =  soup.find_all('div', class_= 'view-content')[0].find_all('a')
    record = [[row['href']] + row.text.split('\n')[-2:] for row in rows]
    return record

In [4]:
df[['records']] = df['pages'].parallel_apply(extract_urls)

In [5]:
df = df.explode('records').reset_index(drop=True)
df[['link', 'title','date']] = df.records.apply(pd.Series)
df = df.iloc[:,2:]
df.link = 'https://www.rescue.org'+ df.link.values
df.head()

,link,title,date
0,https://www.rescue.org/press-release/irc-immigration-bill-rebuilds-spirit-welcome-us-immigration-policy,IRC: Immigration bill rebuilds the spirit of welcome in U.S. immigration policy,"February 19, 2021"
1,https://www.rescue.org/press-release/irc-joins-firstrespondersfirst-initiative-help-humanitarian-staff-manage-psychological,IRC joins #FirstRespondersFirst initiative to help humanitarian staff manage the psychological toll of responding to COVID-19 on the frontlines,"February 18, 2021"
2,https://www.rescue.org/press-release/how-do-we-address-treatment-black-immigrants-us-asylum-system,How do we address the treatment of Black immigrants in the US asylum system?,"February 16, 2021"
3,https://www.rescue.org/press-release/new-ebola-outbreaks-emerge-drc-and-guinea-irc-calls-swift-action-and-funding-frontline,"As new Ebola outbreaks emerge in the DRC and Guinea, the IRC calls for swift action and funding for frontline aid agencies to stop the spread","February 15, 2021"
4,https://www.rescue.org/press-release/irc-receives-178m-usaid-grant-scale-health-services-liberia,IRC receives $17.8m USAID grant to scale up health services in Liberia,"February 9, 2021"


In [6]:
def extract_text(url):
    coverpage = requests.get(url).content
    soup = BeautifulSoup(coverpage, 'html.parser')
    
    main_html = soup.find_all('div', typeof="sioc:Item foaf:Document")[0]

    dateline = soup.find_all('span',class_= 'dateline')

    if len(dateline):
        dateline[0].replace_with('')
    
    paragraphs = main_html.find_all('p', recursive= False)
    text = '\n'.join([p.text for p in paragraphs])
    return pd.Series([str(main_html), text])

In [7]:
df[['html','text']]=None

In [10]:
df[['html','text']]=\
df['link'].parallel_apply(extract_text).values

In [11]:
df

,link,title,date,html,text
0,https://www.rescue.org/press-release/irc-immigration-bill-rebuilds-spirit-welcome-us-immigration-policy,IRC: Immigration bill rebuilds the spirit of welcome in U.S. immigration policy,"February 19, 2021","<div about=""/press-release/irc-immigration-bill-rebuilds-spirit-welcome-us-immigration-policy"" class=""ds-1col node node-press-release full body-format view-mode-full"" typeof=""sioc:Item foaf:Docume...","The IRC welcomes the introduction of the ‘U.S. Citizenship Act of 2021’ immigration bill to Congress today, helping build back humanity and competence to the U.S. immigration system.\nThe immigrat..."
1,https://www.rescue.org/press-release/irc-joins-firstrespondersfirst-initiative-help-humanitarian-staff-manage-psychological,IRC joins #FirstRespondersFirst initiative to help humanitarian staff manage the psychological toll of responding to COVID-19 on the frontlines,"February 18, 2021","<div about=""/press-release/irc-joins-firstrespondersfirst-initiative-help-humanitarian-staff-manage-psychological"" class=""ds-1col node node-press-release full body-format view-mode-full"" typeof=""s...","Today the International Rescue Committee announced a partnership with #FirstRespondersFirst, an initiative of Harvard T.H. Chan School of Public Health, Thrive Global, CAA Foundation, and strategi..."
2,https://www.rescue.org/press-release/how-do-we-address-treatment-black-immigrants-us-asylum-system,How do we address the treatment of Black immigrants in the US asylum system?,"February 16, 2021","<div about=""/press-release/how-do-we-address-treatment-black-immigrants-us-asylum-system"" class=""ds-1col node node-press-release full body-format view-mode-full"" typeof=""sioc:Item foaf:Document"">\...","The US asylum system is under major construction. With hopes of rebuilding and restoring a fair, humane and credible asylum system, the Biden Administration has encountered some initial challenges..."
3,https://www.rescue.org/press-release/new-ebola-outbreaks-emerge-drc-and-guinea-irc-calls-swift-action-and-funding-frontline,"As new Ebola outbreaks emerge in the DRC and Guinea, the IRC calls for swift action and funding for frontline aid agencies to stop the spread","February 15, 2021","<div about=""/press-release/new-ebola-outbreaks-emerge-drc-and-guinea-irc-calls-swift-action-and-funding-frontline"" class=""ds-1col node node-press-release full body-format view-mode-full"" typeof=""s...","As a fourth case of Ebola is confirmed in North Kivu, Democratic Republic of Congo (DRC), the International Rescue Committee (IRC) is extremely concerned that another Ebola outbreak in the area wi..."
4,https://www.rescue.org/press-release/irc-receives-178m-usaid-grant-scale-health-services-liberia,IRC receives $17.8m USAID grant to scale up health services in Liberia,"February 9, 2021","<div about=""/press-release/irc-receives-178m-usaid-grant-scale-health-services-liberia"" class=""ds-1col node node-press-release full body-format view-mode-full"" typeof=""sioc:Item foaf:Document"">\n<...",IRC Liberia has received a $17.8 million grant from USAID; the United States Agency for International Development to support the government of Liberia’s National Community Health Program. Approxim...
...,...,...,...,...,...
700,https://www.rescue.org/press-release/young-women-kenya-need-help-finding-jobs-and-starting-own-businesses-says-irc,"Young women in Kenya need help finding jobs and starting own businesses, says IRC","October 10, 2013","<div about=""/press-release/young-women-kenya-need-help-finding-jobs-and-starting-own-businesses-says-irc"" class=""ds-1col node node-press-release full body-format view-mode-full"" typeof=""sioc:Item ...","As the world marks International Day of the Girl Child on Friday, October 11th, the International Rescue Committee in Kenya says women and girls must be given more economic support so they can not..."
701,https://www.rescue.org/press-release/usaid-and-international-rescue-committee-launch-160-million

In [12]:
df.to_csv('rescue.csv', index=False)

In [2]:
df= pd.read_csv('rescue.csv', header=0)
df

,link,title,date,html,text
0,https://www.rescue.org/press-release/irc-immigration-bill-rebuilds-spirit-welcome-us-immigration-policy,IRC: Immigration bill rebuilds the spirit of welcome in U.S. immigration policy,"February 19, 2021","<div about=""/press-release/irc-immigration-bill-rebuilds-spirit-welcome-us-immigration-policy"" class=""ds-1col node node-press-release full body-format view-mode-full"" typeof=""sioc:Item foaf:Docume...","The IRC welcomes the introduction of the ‘U.S. Citizenship Act of 2021’ immigration bill to Congress today, helping build back humanity and competence to the U.S. immigration system.\nThe immigrat..."
1,https://www.rescue.org/press-release/irc-joins-firstrespondersfirst-initiative-help-humanitarian-staff-manage-psychological,IRC joins #FirstRespondersFirst initiative to help humanitarian staff manage the psychological toll of responding to COVID-19 on the frontlines,"February 18, 2021","<div about=""/press-release/irc-joins-firstrespondersfirst-initiative-help-humanitarian-staff-manage-psychological"" class=""ds-1col node node-press-release full body-format view-mode-full"" typeof=""s...","Today the International Rescue Committee announced a partnership with #FirstRespondersFirst, an initiative of Harvard T.H. Chan School of Public Health, Thrive Global, CAA Foundation, and strategi..."
2,https://www.rescue.org/press-release/how-do-we-address-treatment-black-immigrants-us-asylum-system,How do we address the treatment of Black immigrants in the US asylum system?,"February 16, 2021","<div about=""/press-release/how-do-we-address-treatment-black-immigrants-us-asylum-system"" class=""ds-1col node node-press-release full body-format view-mode-full"" typeof=""sioc:Item foaf:Document"">\...","The US asylum system is under major construction. With hopes of rebuilding and restoring a fair, humane and credible asylum system, the Biden Administration has encountered some initial challenges..."
3,https://www.rescue.org/press-release/new-ebola-outbreaks-emerge-drc-and-guinea-irc-calls-swift-action-and-funding-frontline,"As new Ebola outbreaks emerge in the DRC and Guinea, the IRC calls for swift action and funding for frontline aid agencies to stop the spread","February 15, 2021","<div about=""/press-release/new-ebola-outbreaks-emerge-drc-and-guinea-irc-calls-swift-action-and-funding-frontline"" class=""ds-1col node node-press-release full body-format view-mode-full"" typeof=""s...","As a fourth case of Ebola is confirmed in North Kivu, Democratic Republic of Congo (DRC), the International Rescue Committee (IRC) is extremely concerned that another Ebola outbreak in the area wi..."
4,https://www.rescue.org/press-release/irc-receives-178m-usaid-grant-scale-health-services-liberia,IRC receives $17.8m USAID grant to scale up health services in Liberia,"February 9, 2021","<div about=""/press-release/irc-receives-178m-usaid-grant-scale-health-services-liberia"" class=""ds-1col node node-press-release full body-format view-mode-full"" typeof=""sioc:Item foaf:Document"">\n<...",IRC Liberia has received a $17.8 million grant from USAID; the United States Agency for International Development to support the government of Liberia’s National Community Health Program. Approxim...
...,...,...,...,...,...
700,https://www.rescue.org/press-release/young-women-kenya-need-help-finding-jobs-and-starting-own-businesses-says-irc,"Young women in Kenya need help finding jobs and starting own businesses, says IRC","October 10, 2013","<div about=""/press-release/young-women-kenya-need-help-finding-jobs-and-starting-own-businesses-says-irc"" class=""ds-1col node node-press-release full body-format view-mode-full"" typeof=""sioc:Item ...","As the world marks International Day of the Girl Child on Friday, October 11th, the International Rescue Committee in Kenya says women and girls must be given more economic support so they can not..."
701,https://www.rescue.org/press-release/usaid-and-international-rescue-committee-launch-160-million

In [16]:
from newspaper import Article
def get_news_text(html):
    a = Article('')
    a.set_html(html)
    a.parse()
    text= a.text
    return text

pandarallel.initialize(progress_bar=True)
df[['news']] = df['html'].parallel_apply(get_news_text).values

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [19]:
df_sample = df.sample(n=50).reset_index(drop=True)
for i,j in zip(df_sample.link, df_sample.news):
    print(i)
    print(j)
    print('\n')

https://www.rescue.org/press-release/international-rescue-committee-announces-first-ever-emergency-appeal-refugees-us
The International Rescue Committee (IRC) today announced its first-ever emergency appeal focused exclusively on aiding refugees already in the United States and supporting the critical services IRC provides to ensure the newest Americans – most of which are women and children – can reclaim their lives.

The $5 million initial appeal will cover anticipated funding gaps tied to the Trump Administration’s Executive Order banning refugees for 120 days, and Syrian refugees indefinitely. This emergency funding will support those who look to the IRC’s 29 offices across the United States for necessary and immediate aid on arrival and beyond— including housing, cultural orientation, health care, education, employment, and immigration services. Contributions will also support the work of IRC’s dedicated field teams who are crucial to ensuring clients are provided with a solid fou

In [24]:
# df = df[['link','title','date','news']]
# df.date = pd.to_datetime(df.date)
df.columns = ['url','title','date','text']
df = df.sort_values('date')
df

,url,title,date,text
704,https://www.rescue.org/press-release/syria-displacement-crisis-worsens-protracted-humanitarian-emergency-looms,Syria displacement crisis worsens as protracted humanitarian emergency looms,2013-01-14,"Nearly two years into Syria’s civil war, the region faces a staggering humanitarian disaster, requiring the international community to urgently scale up planning and funding for what is certain to..."
703,https://www.rescue.org/press-release/kenya-civil-society-calls-government-end-abuse-refugees,Kenya: Civil society calls on government to end abuse of refugees,2013-01-22,Several civil society groups on Tuesday demanded that the Kenyan government end police harassment and abuse of refugees and protect the basic human rights of all refugees and Kenyan citizens.\n\n“...
702,https://www.rescue.org/press-release/race-against-time-aid-new-darfur-refugees-chad-rains-begin,Race against time to aid new Darfur refugees in Chad before rains begin,2013-05-21,International Rescue Committee teams in Chad are trucking emergency water supplies into the country’s Tissi region to aid thousands of refugees who fled there in recent months to escape worsening ...
701,https://www.rescue.org/press-release/usaid-and-international-rescue-committee-launch-160-million-pakistan-reading-project,USAID and International Rescue Committee to launch $160 million Pakistan Reading Project,2013-09-07,"The U.S. Agency for International Development has awarded the International Rescue Committee $160 million to implement the Pakistan Reading Project, an extensive education program in Pakistan aime..."
700,https://www.rescue.org/press-release/young-women-kenya-need-help-finding-jobs-and-starting-own-businesses-says-irc,"Young women in Kenya need help finding jobs and starting own businesses, says IRC",2013-10-10,"As the world marks International Day of the Girl Child on Friday, October 11th, the International Rescue Committee in Kenya says women and girls must be given more economic support so they can not..."
...,...,...,...,...
4,https://www.rescue.org/press-release/irc-receives-178m-usaid-grant-scale-health-services-liberia,IRC receives $17.8m USAID grant to scale up health services in Liberia,2021-02-09,IRC Liberia has received a $17.8 million grant from USAID; the United States Agency for International Development to support the government of Liberia’s National Community Health Program. Approxim...
3,https://www.rescue.org/press-release/new-ebola-outbreaks-emerge-drc-and-guinea-irc-calls-swift-action-and-funding-frontline,"As new Ebola outbreaks emerge in the DRC and Guinea, the IRC calls for swift action and funding for frontline aid agencies to stop the spread",2021-02-15,"As a fourth case of Ebola is confirmed in North Kivu, Democratic Republic of Congo (DRC), the International Rescue Committee (IRC) is extremely concerned that another Ebola outbreak in the area wi..."
2,https://www.rescue.org/press-release/how-do-we-address-treatment-black-immigrants-us-asylum-system,How do we address the treatment of Black immigrants in the US asylum system?,2021-02-16,"The US asylum system is under major construction. With hopes of rebuilding and restoring a fair, humane and credible asylum system, the Biden Administration has encountered some initial challenges..."
1,https://www.rescue.org/press-release/irc-joins-firstrespondersfirst-initiative-help-humanitarian-staff-manage-psychological,IRC joins #FirstRespondersFirst initiative to help humanitarian staff manage the psychological toll of responding to COVID-19 on the frontlines,2021-02-18,"Today the International Rescue Committee announced a partnership with #FirstRespondersFirst, an initiative of Harvard T.H. Chan School of Public Health, Thrive Global, CAA Foundation, and strategi..."


In [25]:
df.to_csv('rescue.csv', index=False)

In [2]:
df= pd.read_csv('rescue.csv', header=0)
df.sort_values('date')

,url,title,date,text
0,https://www.rescue.org/press-release/syria-displacement-crisis-worsens-protracted-humanitarian-emergency-looms,Syria displacement crisis worsens as protracted humanitarian emergency looms,2013-01-14,"Nearly two years into Syria’s civil war, the region faces a staggering humanitarian disaster, requiring the international community to urgently scale up planning and funding for what is certain to..."
1,https://www.rescue.org/press-release/kenya-civil-society-calls-government-end-abuse-refugees,Kenya: Civil society calls on government to end abuse of refugees,2013-01-22,Several civil society groups on Tuesday demanded that the Kenyan government end police harassment and abuse of refugees and protect the basic human rights of all refugees and Kenyan citizens.\n\n“...
2,https://www.rescue.org/press-release/race-against-time-aid-new-darfur-refugees-chad-rains-begin,Race against time to aid new Darfur refugees in Chad before rains begin,2013-05-21,International Rescue Committee teams in Chad are trucking emergency water supplies into the country’s Tissi region to aid thousands of refugees who fled there in recent months to escape worsening ...
3,https://www.rescue.org/press-release/usaid-and-international-rescue-committee-launch-160-million-pakistan-reading-project,USAID and International Rescue Committee to launch $160 million Pakistan Reading Project,2013-09-07,"The U.S. Agency for International Development has awarded the International Rescue Committee $160 million to implement the Pakistan Reading Project, an extensive education program in Pakistan aime..."
4,https://www.rescue.org/press-release/young-women-kenya-need-help-finding-jobs-and-starting-own-businesses-says-irc,"Young women in Kenya need help finding jobs and starting own businesses, says IRC",2013-10-10,"As the world marks International Day of the Girl Child on Friday, October 11th, the International Rescue Committee in Kenya says women and girls must be given more economic support so they can not..."
...,...,...,...,...
700,https://www.rescue.org/press-release/irc-receives-178m-usaid-grant-scale-health-services-liberia,IRC receives $17.8m USAID grant to scale up health services in Liberia,2021-02-09,IRC Liberia has received a $17.8 million grant from USAID; the United States Agency for International Development to support the government of Liberia’s National Community Health Program. Approxim...
701,https://www.rescue.org/press-release/new-ebola-outbreaks-emerge-drc-and-guinea-irc-calls-swift-action-and-funding-frontline,"As new Ebola outbreaks emerge in the DRC and Guinea, the IRC calls for swift action and funding for frontline aid agencies to stop the spread",2021-02-15,"As a fourth case of Ebola is confirmed in North Kivu, Democratic Republic of Congo (DRC), the International Rescue Committee (IRC) is extremely concerned that another Ebola outbreak in the area wi..."
702,https://www.rescue.org/press-release/how-do-we-address-treatment-black-immigrants-us-asylum-system,How do we address the treatment of Black immigrants in the US asylum system?,2021-02-16,"The US asylum system is under major construction. With hopes of rebuilding and restoring a fair, humane and credible asylum system, the Biden Administration has encountered some initial challenges..."
703,https://www.rescue.org/press-release/irc-joins-firstrespondersfirst-initiative-help-humanitarian-staff-manage-psychological,IRC joins #FirstRespondersFirst initiative to help humanitarian staff manage the psychological toll of responding to COVID-19 on the frontlines,2021-02-18,"Today the International Rescue Committee announced a partnership with #FirstRespondersFirst, an initiative of Harvard T.H. Chan School of Public Health, Thrive Global, CAA Foundation, and strategi..."


In [26]:
df

,url,title,date,text
704,https://www.rescue.org/press-release/syria-displacement-crisis-worsens-protracted-humanitarian-emergency-looms,Syria displacement crisis worsens as protracted humanitarian emergency looms,2013-01-14,"Nearly two years into Syria’s civil war, the region faces a staggering humanitarian disaster, requiring the international community to urgently scale up planning and funding for what is certain to..."
703,https://www.rescue.org/press-release/kenya-civil-society-calls-government-end-abuse-refugees,Kenya: Civil society calls on government to end abuse of refugees,2013-01-22,Several civil society groups on Tuesday demanded that the Kenyan government end police harassment and abuse of refugees and protect the basic human rights of all refugees and Kenyan citizens.\n\n“...
702,https://www.rescue.org/press-release/race-against-time-aid-new-darfur-refugees-chad-rains-begin,Race against time to aid new Darfur refugees in Chad before rains begin,2013-05-21,International Rescue Committee teams in Chad are trucking emergency water supplies into the country’s Tissi region to aid thousands of refugees who fled there in recent months to escape worsening ...
701,https://www.rescue.org/press-release/usaid-and-international-rescue-committee-launch-160-million-pakistan-reading-project,USAID and International Rescue Committee to launch $160 million Pakistan Reading Project,2013-09-07,"The U.S. Agency for International Development has awarded the International Rescue Committee $160 million to implement the Pakistan Reading Project, an extensive education program in Pakistan aime..."
700,https://www.rescue.org/press-release/young-women-kenya-need-help-finding-jobs-and-starting-own-businesses-says-irc,"Young women in Kenya need help finding jobs and starting own businesses, says IRC",2013-10-10,"As the world marks International Day of the Girl Child on Friday, October 11th, the International Rescue Committee in Kenya says women and girls must be given more economic support so they can not..."
...,...,...,...,...
4,https://www.rescue.org/press-release/irc-receives-178m-usaid-grant-scale-health-services-liberia,IRC receives $17.8m USAID grant to scale up health services in Liberia,2021-02-09,IRC Liberia has received a $17.8 million grant from USAID; the United States Agency for International Development to support the government of Liberia’s National Community Health Program. Approxim...
3,https://www.rescue.org/press-release/new-ebola-outbreaks-emerge-drc-and-guinea-irc-calls-swift-action-and-funding-frontline,"As new Ebola outbreaks emerge in the DRC and Guinea, the IRC calls for swift action and funding for frontline aid agencies to stop the spread",2021-02-15,"As a fourth case of Ebola is confirmed in North Kivu, Democratic Republic of Congo (DRC), the International Rescue Committee (IRC) is extremely concerned that another Ebola outbreak in the area wi..."
2,https://www.rescue.org/press-release/how-do-we-address-treatment-black-immigrants-us-asylum-system,How do we address the treatment of Black immigrants in the US asylum system?,2021-02-16,"The US asylum system is under major construction. With hopes of rebuilding and restoring a fair, humane and credible asylum system, the Biden Administration has encountered some initial challenges..."
1,https://www.rescue.org/press-release/irc-joins-firstrespondersfirst-initiative-help-humanitarian-staff-manage-psychological,IRC joins #FirstRespondersFirst initiative to help humanitarian staff manage the psychological toll of responding to COVID-19 on the frontlines,2021-02-18,"Today the International Rescue Committee announced a partnership with #FirstRespondersFirst, an initiative of Harvard T.H. Chan School of Public Health, Thrive Global, CAA Foundation, and strategi..."


In [12]:
df.text[617	]=\
"New York, NY, August 18, 2016 — The below statement can be attributed to Ciaran Donnelly, Senior Vice President of International Programs at the International Rescue Committee:\n"+\
"World Humanitarian Day is an annual tribute to the tens of thousands of brave men and women who have dedicated their lives to humanitarian service, working tirelessly to alleviate suffering worldwide. Their service has indeed never been more necessary: with 130 million people in need of life-saving humanitarian assistance, and an unprecedented 65 million people – the populations of New Zealand, Canada, and Australia combined – displaced from their homes by violence and conflict, the demands on humanitarian action are greater than ever.\n"+\
"And yet, the critical work of humanitarian aid workers remains under threat. Recent attacks on humanitarian staff, most of whom are national, and their facilities highlight the ongoing and increasing dangers they face in carrying out their vital mission. These attacks- from hospitals in Syria, Afghanistan and Yemen to deadly raids in South Sudan- are not only serious violations of international humanitarian law, but a repudiation of the values and aims of humanitarianism. Attacks on aid workers are not just a crime in and of themselves - their impact extends far beyond, penalizing those who depend on this assistance: the world's most vulnerable people.\n"+\
"One aid worker or facility targeted in conflict is one too many. On this World Humanitarian Day, the IRC calls for the full respect of international humanitarian law by all parties to conflict worldwide, as well as the full respect of the sanctity of humanitarian action, allowing for safe, secure and unimpeded access to those in urgent need."
"World Humanitarian Day must serve as a reminder of our solidarity with the millions in need around the world, and our respect of those working on the front lines of the greatest humanitarian challenge of our time."

'World Humanitarian Day must serve as a reminder of our solidarity with the millions in need around the world, and our respect of those working on the front lines of the greatest humanitarian challenge of our time.'

In [18]:
df.text.str.len().describe()

count      705.000000
mean      3556.459574
std       3619.610841
min        313.000000
25%       1872.000000
50%       2867.000000
75%       3984.000000
max      33933.000000
Name: text, dtype: float64

In [17]:
df.news.str.len().describe()

count      705.000000
mean      4054.492199
std       3981.688259
min        405.000000
25%       2173.000000
50%       3140.000000
75%       4645.000000
max      35833.000000
Name: news, dtype: float64

In [3]:
df[df.text.duplicated()]

,link,title,date,html,text
666,https://www.rescue.org/press-release/increase-airstrikes-and-ongoing-sieges-making-mockery-international-efforts-help,"Increase in airstrikes and ongoing sieges “making a mockery” of international efforts to help Syrians, says aid agency","February 11, 2016","<div about=""/press-release/international-rescue-committee-country-director-greece-reacts-todays-events-moria"" class=""ds-1col node node-press-release full body-format view-mode-full"" typeof=""sioc:I...","In reaction to today's events at Moria on the island of Lesbos, the IRC's Country Director in Greece, Panos Navrozidis, said:\n""Events at Moria highlight the level of frustration there. Many of th..."
668,https://www.rescue.org/press-release/million-jobs-needed-syrian-refugees-make-life-worth-living-region-says-humanitarian,"A million jobs needed for Syrian refugees to make ‘life worth living’ in the region, says humanitarian agency","February 1, 2016","<div about=""/press-release/million-jobs-needed-syrian-refugees-make-life-worth-living-region-says-humanitarian"" class=""ds-1col node node-press-release full body-format view-mode-full"" typeof=""sioc...","The international community must massively invest in the economies of countries neighbouring Syria to create conditions for a million Syrians to find work and support themselves, says the Internat..."
